Limpieza de datos Rooms

In [20]:
import pandas as pd
import numpy as np
import re 

In [3]:
data_location = "./Data/properati.csv"
data = pd.read_csv(data_location, sep=",")
print(data.dtypes)
data.head(3)

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [184]:
data["rooms_completo"] =data["rooms"]
data_room_null_description = data.description.astype(str).str.lower()
display(data_room_null_description)

0         2 ambientes tipo casa planta baja por pasillo,...
1         venta de departamento en décimo piso al frente...
2         2 ambientes 3er piso lateral living comedor am...
3         ph 3 ambientes con patio. hay 3 deptos en lote...
4         departamento con fantástica iluminación natura...
                                ...                        
121215    torre forum alcorta - máxima categoría.impecab...
121216    excelente e impecable casa en venta en las lom...
121217    venta departamento ambiente divisible a estren...
121218    2 amb al contrafrente,  luminoso. el departame...
121219    departamento de 77 m2 apto profesional. edific...
Name: description, Length: 121220, dtype: object

In [185]:
data_room_mask = data["rooms_completo"].isnull()
data["rooms_completo"].isnull().value_counts()

True     73830
False    47390
Name: rooms_completo, dtype: int64

In [ ]:
df['set_of_numbers'].apply(lambda x: 'True' if x <= 4 else 'False')

seteamos los que son monoambientes

In [209]:
patt_room = r"(mono ambiente|monoambiente)" #Amb
patt_room_regex = re.compile(patt_room)
patt_room_regex 
data_room_match = data_room_null_description.apply(lambda x:  x if x is np.NaN else patt_room_regex.search(x))
data_room_match_mask = data_room_match.notnull()
data_room_mask = data["rooms_completo"].isnull()


#data_room_match[121217].string
data.loc[data_room_match_mask,"rooms_completo"] = 1
# indices_mask_notnull= data_room_match.notnull()
data["rooms_completo"].isnull().value_counts()

True     69213
False    52007
Name: rooms_completo, dtype: int64

Ahora vamos con los que pertenecen a varios ambientes, se parte de la premisa que 2 ambientes tiene al menos una habitación y sucesivamente. 

In [162]:
data_room_mask= data["rooms_completo"].isnull() #.value_counts()
data_room_null_description = data[data_rooms_mask].description.astype(str).str.lower()
data_room_null_description

0         2 ambientes tipo casa planta baja por pasillo,...
1         venta de departamento en décimo piso al frente...
2         2 ambientes 3er piso lateral living comedor am...
3         ph 3 ambientes con patio. hay 3 deptos en lote...
4         departamento con fantástica iluminación natura...
                                ...                        
121215    torre forum alcorta - máxima categoría.impecab...
121216    excelente e impecable casa en venta en las lom...
121217    venta departamento ambiente divisible a estren...
121218    2 amb al contrafrente,  luminoso. el departame...
121219    departamento de 77 m2 apto profesional. edific...
Name: description, Length: 73830, dtype: object

In [149]:
patt_rooms = r"([0-9](ambiente)"
patt_rooms_regex = re.compile(patt_room)
patt_rooms_regex 
data_rooms_match = data_rooms_null_description.apply(lambda x:  x if x is np.NaN else patt_rooms_regex.search(x))
indices_mask_notnull= data_rooms_match.notnull()
#data.loc[indices_mask_notnull,"rooms_completo"]=1
indices_mask_notnull.isnull().value_counts()

False    73830
Name: description, dtype: int64